In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
make_hsc_rgb_v5.py
  · 支持 g/r/i 任意 1–3 个波段
  · 自动 WCS 对齐，单/双波段也输出
"""

import re
from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.visualization import make_lupton_rgb
from reproject import reproject_interp
import matplotlib.pyplot as pl

CUT_DIR = Path("./hsc_cutouts")
RGB_DIR = Path("./hsc_rgb_cutouts")
RGB_DIR.mkdir(exist_ok=True)

PAT      = re.compile(r"hsc(?P<tid>\d+)_(?P<b>[gri])\.fits$", re.I)
WEIGHTS  = dict(g=1.8, r=2.0, i=3.0)
LUPTON   = dict(Q=8, stretch=1)

# ---------- 1. 构建 catalog ----------
catalog: Dict[str, Dict[str, Path]] = {}
for fp in CUT_DIR.iterdir():
    m = PAT.match(fp.name)
    if m:
        catalog.setdefault(m["tid"], {})[m["b"].lower()] = fp

tids = sorted(catalog)                           # 不筛波段数，全部处理
print(f"共有 {len(tids)} 个目标，开始合成…")

# ---------- 2. 小工具 ----------
def read_data_wcs(fp: Path):
    with fits.open(fp, memmap=False) as hdul:
        for hdu in hdul:
            if hdu.data is not None and hdu.data.ndim == 2:
                return hdu.data.astype("f4"), WCS(hdu.header)
    raise ValueError("no 2-D image found")

def preprocess(img: np.ndarray) -> np.ndarray:
    img = np.nan_to_num(img, nan=0.0)
    good = img > 0
    if good.sum() < 50:
        return np.zeros_like(img)

    sky = np.percentile(img[good], 0.5)
    hi  = np.percentile(img[good], 99)
    if hi - sky < 1e-3:
        mu  = np.median(img[good])
        sig = 1.4826 * np.median(np.abs(img[good] - mu))
        sky, hi = mu - sig, mu + 5 * sig
    return np.clip((img - sky) / (hi - sky + 1e-6), 0, 1)

# ---------- 3. 主循环 ----------
bad: List[dict] = []

for k, tid in enumerate(tids, 1):
    bands = catalog[tid]                         # {'g': Path, ...}
    out_png = RGB_DIR / f"hsc{tid}_rgb.png"

    # 3-A 读参考波段并确定输出网格
    ref_band = next(b for b in "irg" if b in bands)  # i 优先
    try:
        ref_dat, ref_wcs = read_data_wcs(bands[ref_band])
    except Exception as e:
        bad.append({"tid": tid, "err": f"read_ref_fail: {e}"})
        print(f"[{k}/{len(tids)}] 参考波段读取失败 {tid}: {e}")
        continue

    shape_out = ref_dat.shape
    ch: Dict[str, np.ndarray] = dict.fromkeys("gri")

    # 3-B 逐波段读取并（如需）重投影
    for b in "gri":
        if b in bands:
            try:
                dat, wcs = read_data_wcs(bands[b])
                if b == ref_band:                # 参考波段
                    ch[b] = preprocess(dat) * WEIGHTS[b]
                else:                            # 需要重投影
                    dat_rp, _ = reproject_interp((dat, wcs), ref_wcs,
                                                 shape_out=shape_out,
                                                 order="bilinear")
                    ch[b] = preprocess(dat_rp) * WEIGHTS[b]
            except Exception as e:
                bad.append({"tid": tid, "err": f"{b}_fail: {e}"})
                ch[b] = np.zeros(shape_out, dtype="f4")
                print(f"[{k}/{len(tids)}] {b} 处理失败 {tid}: {e}")
        else:
            ch[b] = np.zeros(shape_out, dtype="f4")  # 缺失 → 0

    # 3-C 根据可用波段数选择保存方式
    n_band = sum((ch[b] > 0).any() for b in "gri")
    try:
        if n_band == 1:
            # 灰度：复制到 3 通道
            gray = next(ch[b] for b in "gri" if (ch[b] > 0).any())
            rgb  = np.dstack([gray, gray, gray])
            pl.imsave(out_png, rgb, origin="lower", cmap="gray")
        else:
            rgb = make_lupton_rgb(ch["i"], ch["r"], ch["g"],
                                  minimum=0, **LUPTON)
            pl.imsave(out_png, rgb, origin="lower")
        print(f"[{k}/{len(tids)}] 生成 {out_png.name}（{n_band} band）")
    except Exception as e:
        bad.append({"tid": tid, "err": f"save_fail: {e}"})
        print(f"[{k}/{len(tids)}] 保存失败 {tid}: {e}")

# ---------- 4. 记录异常 ----------
if bad:
    bad_csv = RGB_DIR / "bad_rgb_fits.csv"
    pd.DataFrame(bad).to_csv(bad_csv, index=False)
    print(f"\n共有 {len(bad)} 个目标跳过或部分失败，已写入 {bad_csv}")
else:
    print("\n全部目标合成完毕，无异常")

共有 285 个目标，开始合成…
[1/285] 生成 hsc1030507957583872_rgb.png（3 band）
[2/285] 生成 hsc1030526211194880_rgb.png（3 band）
[3/285] 生成 hsc1030550353608705_rgb.png（3 band）
[4/285] 生成 hsc1030573330006017_rgb.png（3 band）
[5/285] 生成 hsc2349816348672001_rgb.png（3 band）
[6/285] 生成 hsc2349858451095553_rgb.png（3 band）
[7/285] 生成 hsc2349858560147457_rgb.png（3 band）
[8/285] 生成 hsc2349888679444481_rgb.png（3 band）
[9/285] 生成 hsc2349888733970433_rgb.png（3 band）
[10/285] 生成 hsc2349891657400321_rgb.png（3 band）
[11/285] 生成 hsc2349896627650561_rgb.png（3 band）
[12/285] 生成 hsc2349897944662017_rgb.png（3 band）
[13/285] 生成 hsc2349903170764801_rgb.png（3 band）
[14/285] 生成 hsc2349906786254849_rgb.png（3 band）
[15/285] 生成 hsc2349914600243200_rgb.png（3 band）
[16/285] 生成 hsc2349915866923008_rgb.png（3 band）
[17/285] 生成 hsc2349915866923009_rgb.png（3 band）
[18/285] 生成 hsc2349920623263744_rgb.png（3 band）
[19/285] 生成 hsc2349920623263745_rgb.png（3 band）
[20/285] 生成 hsc2349921910915072_rgb.png（3 band）
[21/285] 生成 hsc2349921910915073_